In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import gc
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool
import seaborn as sns
import joblib
import lightgbm as lgb
from sklearn.inspection import permutation_importance
import os
import wandb
import time

In [2]:
def weighted_r2_metric(y_true, y_pred, weights=None):
    if weights is None:
        weights = np.ones_like(y_true)

    numerator = np.sum(weights * (y_true - y_pred)**2)

    y_weighted_mean = np.sum(weights * y_true) / np.sum(weights)
    denominator = np.sum(weights * (y_true - y_weighted_mean)**2)

    score = 1 - (numerator / denominator)
    return 'weighted_r2', score, True

responder_cols = [f"responder_{col}" for col in range(9)]

In [3]:
df_list = []
for part_in in ["6","7","8","9"]:
    part_id = part_in
    data_dir = f"train.parquet/partition_id={part_id}/part-0.parquet"
    df_list.append(pl.read_parquet(data_dir))
    gc.collect()

In [4]:
train_df = pl.concat(df_list)
del df_list
gc.collect()

0

In [6]:
def create_daily_stats(train_df, target_col):
    daily_avg = (
        train_df
        .group_by(["symbol_id", "date_id"], maintain_order=True)
        .agg([pl.col(target_col).mean().alias("daily_avg"), 
            pl.col(target_col).std().alias("daily_std"),
            pl.col(target_col).min().alias("daily_min"),
            pl.col(target_col).max().alias("daily_max"),
            pl.col(target_col).median().alias("daily_median"),
            pl.col(target_col).skew().alias("daily_skew"),
            pl.col(target_col).kurtosis().alias("daily_kurtosis"),
            pl.col(target_col).last().alias("last_value"),
            pl.col(target_col).first().alias("first_value"),
            (pl.col(target_col).max() - pl.col(target_col).min()).alias("daily_range"),
            pl.col(target_col).sum().alias("target_sum"),
            pl.col(target_col).count().alias("daily_count")
            ]
            )
    )
    daily_avg = daily_avg.sort(["symbol_id", "date_id"])

    columns_to_shift = ["daily_avg", "daily_std", "daily_min", "daily_max", "daily_median", "daily_skew", "daily_kurtosis", "last_value", "first_value",
    "daily_range", "target_sum", "date_id", "daily_count"]

    daily_avg = daily_avg.with_columns([
        pl.col(col_name)
        .shift(1)
        .over("symbol_id")
        .alias(f"lag_1_{col_name}_{target_col}")
        for col_name in columns_to_shift
        ])

    s1 = [f"lag_1_{col_name}_{target_col}" for col_name in columns_to_shift if col_name != "date_id"]

    selected_cols = ["symbol_id","date_id"] + s1
    daily_avg = daily_avg.select(selected_cols)

    train_df = train_df.join(daily_avg,
              on=["symbol_id", "date_id"],
              how="left")

    return train_df

def create_daily_stats2(train_df, target_col):
    daily_avg = (
        train_df
        .group_by(["symbol_id", "date_id"], maintain_order = True)
        .agg([pl.col(target_col).mean().alias("daily_avg"), 
            pl.col(target_col).std().alias("daily_std"),
            pl.col(target_col).min().alias("daily_min"),
            pl.col(target_col).max().alias("daily_max"),
            pl.col(target_col).median().alias("daily_median"),
            pl.col(target_col).last().alias("last_value"),
            (pl.col(target_col).max() - pl.col(target_col).min()).alias("daily_range"),
            pl.col(target_col).sum().alias("target_sum"),
            ]
            )
    )
    daily_avg = daily_avg.sort(["symbol_id", "date_id"])

    columns_to_shift = ["daily_avg", "daily_std", "daily_min", "daily_max", "daily_median", "last_value","daily_range", "target_sum"]

    daily_avg = daily_avg.with_columns([
        pl.col(col_name)
        .shift(1)
        .over("symbol_id")
        .alias(f"lag_1_{col_name}_{target_col}")
        for col_name in columns_to_shift
    ])


    s1 = [f"lag_1_{col_name}_{target_col}" for col_name in columns_to_shift if col_name != "date_id"]

    selected_cols = ["symbol_id","date_id"] + s1
    daily_avg = daily_avg.select(selected_cols)

    train_df = train_df.join(daily_avg,
              on=["symbol_id", "date_id"],
              how="left")

    train_df = train_df.sort(["date_id", "time_id", "symbol_id"])
    gc.collect()
    return train_df


In [7]:
%%time
for col_name in ["responder_6"]:
    train_df = create_daily_stats(train_df, col_name)
gc.collect()

CPU times: user 7.1 s, sys: 2.25 s, total: 9.35 s
Wall time: 605 ms


0

In [8]:
%%time
for col_name in ["responder_0","responder_1","responder_2","responder_3","responder_4","responder_5","responder_7","responder_8"]:
    train_df = create_daily_stats2(train_df, col_name)
gc.collect()

CPU times: user 2min 17s, sys: 1min 57s, total: 4min 15s
Wall time: 16.7 s


0

In [9]:
train_df = train_df.with_columns([
    (2 * np.pi * pl.col("time_id") / 967).sin().alias("sin_time_id").cast(pl.Float32),
    (2 * np.pi * pl.col("time_id") / 967).cos().alias("cos_time_id").cast(pl.Float32),
])
gc.collect()

0

In [10]:
%%time
default_features = [f"feature_{idx:02d}" for idx in range(79)]
train_df = train_df.with_columns(null_count = pl.sum_horizontal([pl.col(col).is_null() for col in default_features]))
gc.collect()

CPU times: user 1.99 s, sys: 4.37 s, total: 6.36 s
Wall time: 525 ms


0

In [11]:
polars_train_df = train_df
train_df = polars_train_df.to_pandas()
del polars_train_df
gc.collect()

0

In [12]:
train_df["symbol_id"] = train_df["symbol_id"].astype("category")
train_df["feature_10"] = train_df["feature_10"].astype("category")
train_df["feature_11"] = train_df["feature_11"].astype("category")
train_df["feature_09"] = train_df["feature_09"].astype("category")
gc.collect()

0

In [13]:
X_train = train_df[train_df["date_id"] < 1550].copy()
X_test = train_df[train_df["date_id"] >= 1550].copy()
del train_df
gc.collect()

0

In [14]:
y_train = X_train["responder_6"].copy()
y_test = X_test["responder_6"].copy()
test_weights = X_test["weight"].values.copy()
train_weights = X_train["weight"].values.copy()
X_train.drop(columns = responder_cols, axis=1, inplace = True, errors = "ignore")
X_test.drop(columns = responder_cols, axis=1, inplace = True, errors = "ignore")
X_train.drop(columns = ["weight"], axis = 1, inplace = True)
X_test.drop(columns = ["weight"], axis = 1, inplace = True)
gc.collect()

0

In [19]:
feature_cols = [col for col in X_train.columns if col not in ["date_id", "weight"]]
len(feature_cols)

174

In [20]:
feature_cols = [col for col in feature_cols if col not in responder_cols]
len(feature_cols)

174

In [16]:
X_train.drop(columns=[col for col in X_train.columns if col not in feature_cols], inplace=True)
X_test.drop(columns=[col for col in X_test.columns if col not in feature_cols], inplace=True)
gc.collect()

0

In [17]:
%%time
short_id = int(time.time()) % 100_000
cb_params = {"random_seed" : 16,
            "iterations" : 1_500,
            "task_type" : "GPU",
            "objective" : "RMSE",
            "eval_metric" : "R2", 
            "cat_features" : ["symbol_id", "feature_09","feature_10","feature_11"],
            "use_best_model" : False,
            "learning_rate" : 0.01,
            "gpu_ram_part" : 0.9,
            "one_hot_max_size" : 64,
            "border_count" : 128,
            "max_depth" : 6}

exp_name = f"cb_run_{short_id}"

train_data = Pool(X_train, y_train, weight = train_weights, cat_features = ["symbol_id", "feature_09","feature_10","feature_11"])
test_data = Pool(X_test, y_test, weight = test_weights, cat_features = ["symbol_id", "feature_09","feature_10","feature_11"])
cb_model = CatBoostRegressor(**cb_params)

wandb.init(
    project="js_catboost", 
    name=exp_name,  
    tags = ["exp", "cb_v21"],
    config={
        "max_depth" : cb_params["max_depth"],
        "random_seed": cb_params["random_seed"],
        "iterations": cb_params["iterations"],
        "objective": cb_params["objective"],
        "one_hot_max_size" : cb_params["one_hot_max_size"],
        "learning_rate" : cb_params["learning_rate"],
        "border_count" : cb_params["border_count"],
        "cat_features" : cb_params["cat_features"],
        "feature_cols" : train_data.get_feature_names(),
    },
)
wandb.run.notes = "Yeni Featurelar ile deneme, border_count 255 deneme"

cb_model.fit(train_data, eval_set = test_data, verbose = 100)

y_pred_cb = np.clip(cb_model.predict(test_data), -5, 5)
y_pred_train_cb = np.clip(cb_model.predict(train_data), -5, 5)

r_score_test = r2_score(test_data.get_label(), y_pred_cb, sample_weight = test_data.get_weight())
r_score_train = r2_score(train_data.get_label(), y_pred_train_cb, sample_weight = train_data.get_weight())
wandb.run.summary["w_test_r2"] = r_score_test
wandb.run.summary["w_train_r2"] = r_score_train
eval_metrics_train = cb_model.eval_metrics(train_data, metrics = ["R2", "RMSE"], eval_period = 100, thread_count = 16)
eval_metrics_test = cb_model.eval_metrics(test_data, metrics = ["R2", "RMSE"], eval_period = 100, thread_count = 16)


for iteration in range(len(eval_metrics_train['R2'])):
    metrics_to_log = {
        "iteration": (iteration) * 100, 
        "train_r2": eval_metrics_train['R2'][iteration],
        "train_rmse": eval_metrics_train['RMSE'][iteration],
        "test_r2": eval_metrics_test['R2'][iteration],
        "test_rmse": eval_metrics_test['RMSE'][iteration],
    }
    
    wandb.log(metrics_to_log)

cb_model.save_model(os.path.join(wandb.run.dir, f"cb_{short_id}.cbm"))

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: turkenm. Use `wandb login --relogin` to force relogin


Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0001346	test: 0.0000680	best: 0.0000680 (0)	total: 299ms	remaining: 7m 28s
100:	learn: 0.0069737	test: 0.0036626	best: 0.0036626 (100)	total: 13.3s	remaining: 3m 4s
200:	learn: 0.0099538	test: 0.0049433	best: 0.0049433 (200)	total: 27s	remaining: 2m 54s
300:	learn: 0.0121170	test: 0.0057043	best: 0.0057043 (300)	total: 41s	remaining: 2m 43s
400:	learn: 0.0137773	test: 0.0063140	best: 0.0063140 (400)	total: 55.3s	remaining: 2m 31s
500:	learn: 0.0152272	test: 0.0067473	best: 0.0067473 (500)	total: 1m 9s	remaining: 2m 19s
600:	learn: 0.0164650	test: 0.0071258	best: 0.0071258 (600)	total: 1m 24s	remaining: 2m 6s
700:	learn: 0.0177102	test: 0.0074460	best: 0.0074460 (700)	total: 1m 39s	remaining: 1m 53s
800:	learn: 0.0191937	test: 0.0076965	best: 0.0076965 (800)	total: 1m 54s	remaining: 1m 39s
900:	learn: 0.0204937	test: 0.0079158	best: 0.0079158 (900)	total: 2m 9s	remaining: 1m 25s
1000:	learn: 0.0217412	test: 0.0080999	best: 0.0080999 (1000)	total: 2m 24s	remaining: 1m 11s
110

iteration,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_r2,▁▄▅▆▆▆▇▇▇▇██████
test_rmse,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁
train_r2,▁▃▄▄▅▅▅▆▆▆▇▇▇▇██
train_rmse,█▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁
iteration,1500
test_r2,0.00862
test_rmse,0.79855
train_r2,0.02727
train_rmse,0.81823
w_test_r2,0.00862


CPU times: user 10min 43s, sys: 41.6 s, total: 11min 25s
Wall time: 4min 27s


## Catboost Wandb Sweeps

In [23]:
sweep_config = {
    "method": "grid", 
    "metric": {
        "name": "w_test_r2", 
        "goal": "maximize", 
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.025],
        },
        "max_depth": {
            "values": [5, 6, 7], 
        },
        "border_count": {
            "values": [128, 255], 
        },
        "iterations": {
            "values": [1500],
        },
    },
}

In [24]:
def train():
    wandb.init()

    config = wandb.config

    cb_params = {
        "random_seed": 16,
        "iterations": config.iterations,
        "task_type": "GPU",
        "objective": "RMSE",
        "eval_metric": "R2",
        "cat_features": ["symbol_id", "feature_10"],
        "use_best_model": False,
        "learning_rate": config.learning_rate,
        "gpu_ram_part": 0.9,
        "one_hot_max_size": 64,
        "border_count": config.border_count,
        "max_depth": config.max_depth,
    }

    train_data = Pool(X_train, y_train, weight=train_weights, cat_features=["symbol_id", "feature_10"])
    test_data = Pool(X_test, y_test, weight=test_weights, cat_features=["symbol_id", "feature_10"])

    log_config_update={
        "random_seed": cb_params["random_seed"],
        "objective": cb_params["objective"],
        "one_hot_max_size" : cb_params["one_hot_max_size"],
        "cat_features" : cb_params["cat_features"],
        "feature_cols" : train_data.get_feature_names(),
    }
    wandb.config.update(log_config_update, allow_val_change=True)
    wandb.run.notes = "Data partition 5 eklendi"

    cb_model = CatBoostRegressor(**cb_params)
    cb_model.fit(train_data, eval_set=test_data, verbose=100)

    y_pred_cb = np.clip(cb_model.predict(test_data), -5, 5)
    y_pred_train_cb = np.clip(cb_model.predict(train_data), -5, 5)

    r_score_test = r2_score(test_data.get_label(), y_pred_cb, sample_weight=test_data.get_weight())
    r_score_train = r2_score(train_data.get_label(), y_pred_train_cb, sample_weight=train_data.get_weight())

    wandb.run.summary["w_test_r2"] = r_score_test
    wandb.run.summary["w_train_r2"] = r_score_train

    eval_metrics_train = cb_model.eval_metrics(train_data, metrics=["R2", "RMSE"], eval_period=100, thread_count=16)
    eval_metrics_test = cb_model.eval_metrics(test_data, metrics=["R2", "RMSE"], eval_period=100, thread_count=16)

    for iteration in range(len(eval_metrics_train['R2'])):
        metrics_to_log = {
            "iteration": (iteration) * 100,
            "train_r2": eval_metrics_train['R2'][iteration],
            "train_rmse": eval_metrics_train['RMSE'][iteration],
            "test_r2": eval_metrics_test['R2'][iteration],
            "test_rmse": eval_metrics_test['RMSE'][iteration],
        }
        wandb.log(metrics_to_log)

    cb_model.save_model(os.path.join(wandb.run.dir, f"cb_{wandb.run.name}.cbm"))
    wandb.finish()

In [25]:
sweep_id = wandb.sweep(sweep_config, project="js_catboost")

Create sweep with ID: zevgd9kj
Sweep URL: https://wandb.ai/turkenm/js_catboost/sweeps/zevgd9kj


In [26]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: b2v9mvbh with config:
wandb: 	border_count: 128
wandb: 	iterations: 1500
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0001178	test: 0.0000480	best: 0.0000480 (0)	total: 128ms	remaining: 3m 11s
100:	learn: 0.0067014	test: 0.0031745	best: 0.0031745 (100)	total: 7.64s	remaining: 1m 45s
200:	learn: 0.0097163	test: 0.0044999	best: 0.0044999 (200)	total: 15.4s	remaining: 1m 39s
300:	learn: 0.0117706	test: 0.0052891	best: 0.0052891 (300)	total: 23.4s	remaining: 1m 33s
400:	learn: 0.0136304	test: 0.0057686	best: 0.0057686 (400)	total: 31.5s	remaining: 1m 26s
500:	learn: 0.0153819	test: 0.0061535	best: 0.0061535 (500)	total: 39.9s	remaining: 1m 19s
600:	learn: 0.0170729	test: 0.0064783	best: 0.0064783 (600)	total: 48.1s	remaining: 1m 11s
700:	learn: 0.0185407	test: 0.0068074	best: 0.0068074 (700)	total: 56.5s	remaining: 1m 4s
800:	learn: 0.0198325	test: 0.0070815	best: 0.0070815 (800)	total: 1m 4s	remaining: 56.4s
900:	learn: 0.0211296	test: 0.0072712	best: 0.0072712 (900)	total: 1m 12s	remaining: 48.5s
1000:	learn: 0.0224077	test: 0.0074414	best: 0.0074414 (1000)	total: 1m 21s	remaining: 40.4s
110

iteration,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_r2,▁▄▅▆▆▆▇▇▇▇▇█████
test_rmse,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁
train_r2,▁▃▃▄▄▅▅▆▆▆▇▇▇▇██
train_rmse,█▆▆▅▅▄▄▃▃▃▃▂▂▂▁▁
iteration,1500
test_r2,0.00802
test_rmse,0.79879
train_r2,0.02844
train_rmse,0.8067
w_test_r2,0.00802


wandb: Agent Starting Run: cqddgdh8 with config:
wandb: 	border_count: 128
wandb: 	iterations: 1500
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 6
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: s0twatx6 with config:
wandb: 	border_count: 255
wandb: 	iterations: 1500
wandb: 	learning_rate: 0.025
wandb: 	max_depth: 7
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0003805	test: 0.0001609	best: 0.0001609 (0)	total: 157ms	remaining: 3m 55s
100:	learn: 0.0144187	test: 0.0057204	best: 0.0057204 (100)	total: 9.99s	remaining: 2m 18s
200:	learn: 0.0212640	test: 0.0070248	best: 0.0070248 (200)	total: 20.7s	remaining: 2m 13s
300:	learn: 0.0272199	test: 0.0076743	best: 0.0076743 (300)	total: 31.3s	remaining: 2m 4s
400:	learn: 0.0323261	test: 0.0080597	best: 0.0080597 (400)	total: 41.8s	remaining: 1m 54s
500:	learn: 0.0367515	test: 0.0082351	best: 0.0082351 (500)	total: 52.4s	remaining: 1m 44s
600:	learn: 0.0406881	test: 0.0084805	best: 0.0084805 (600)	total: 1m 3s	remaining: 1m 34s
700:	learn: 0.0441504	test: 0.0085459	best: 0.0085459 (700)	total: 1m 13s	remaining: 1m 24s
800:	learn: 0.0471817	test: 0.0085452	best: 0.0085592 (705)	total: 1m 24s	remaining: 1m 13s
900:	learn: 0.0502010	test: 0.0086532	best: 0.0086532 (900)	total: 1m 34s	remaining: 1m 3s
1000:	learn: 0.0528521	test: 0.0087085	best: 0.0087085 (1000)	total: 1m 45s	remaining: 52.6s


iteration,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_r2,▁▅▆▇▇▇██████████
test_rmse,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁
train_r2,▁▃▃▄▄▅▅▆▆▆▇▇▇███
train_rmse,█▆▆▅▅▄▄▃▃▃▂▂▂▂▁▁
iteration,1500
test_r2,0.009
test_rmse,0.7984
train_r2,0.0648
train_rmse,0.79147
w_test_r2,0.009


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# CB Inference Training

In [18]:
api = wandb.Api()

run = api.run("turkenm/js_catboost/6evp7lfh")

config = run.config
cb_feature_cols = config["feature_cols"]

In [26]:
cb_version = "v18"
joblib.dump(feature_cols, f"model_cols/cb_{cb_version}_model_cols.pkl")
len(feature_cols)

122

In [19]:
len(cb_feature_cols)

122

In [20]:
%%time
cb_version = "v20"
short_id = int(time.time()) % 100_000
cb_params = {"random_seed" : 16,
            "iterations" : 1_500,
            "task_type" : "GPU",
            "objective" : "RMSE",
            "eval_metric" : "R2", 
            "cat_features" : ["symbol_id", "feature_09","feature_10","feature_11"],
            "use_best_model" : False,
            "learning_rate" : 0.01,
            "gpu_ram_part" : 0.9,
            "one_hot_max_size" : 64,
            "border_count" : 128,
            "max_depth" : 6}

exp_name = f"cb_run_{short_id}"

inference_data = Pool(train_df[cb_feature_cols], train_df["responder_6"], weight = train_df["weight"], cat_features = ["symbol_id", "feature_09","feature_10","feature_11"])
cb_model_inference = CatBoostRegressor(**cb_params)

wandb.init(
    project="js_catboost",
    name=exp_name,
    tags = ["inference", f"cb_{cb_version}"],
    config={
        "max_depth" : cb_params["max_depth"],
        "random_seed": cb_params["random_seed"],
        "iterations": cb_params["iterations"],
        "objective": cb_params["objective"],
        "one_hot_max_size" : cb_params["one_hot_max_size"],
        "learning_rate" : cb_params["learning_rate"],
        "border_count" : cb_params["border_count"],
        "cat_features" : cb_params["cat_features"],
        "feature_cols" : inference_data.get_feature_names(),
    },
)
wandb.run.notes = "new features, inference"

cb_model_inference.fit(inference_data, verbose = 100)

y_pred_cb = np.clip(cb_model_inference.predict(inference_data), -5, 5)

r_score_train = r2_score(inference_data.get_label(), y_pred_cb, sample_weight = inference_data.get_weight())

wandb.run.summary["w_train_r2"] = r_score_train

eval_metrics_train = cb_model_inference.eval_metrics(inference_data, metrics = ["R2", "RMSE"], eval_period = 100, thread_count = 16)


for iteration in range(len(eval_metrics_train['R2'])): 
    metrics_to_log = {
        "iteration": (iteration) * 100,
        "train_r2": eval_metrics_train['R2'][iteration],
        "train_rmse": eval_metrics_train['RMSE'][iteration],
    }
    
    wandb.log(metrics_to_log)

cb_model_inference.save_model(os.path.join(wandb.run.dir, f"cb_{short_id}.cbm"))
cb_model_inference.save_model(f"cb_model_results/cb_{cb_version}.cbm")
joblib.dump(cb_feature_cols, f"model_cols/cb_{cb_version}_model_cols.pkl")
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: turkenm. Use `wandb login --relogin` to force relogin


Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0001184	total: 346ms	remaining: 20m 10s
100:	learn: 0.0061118	total: 16.6s	remaining: 9m 19s
200:	learn: 0.0085705	total: 33.8s	remaining: 9m 15s
300:	learn: 0.0104072	total: 51.9s	remaining: 9m 11s
400:	learn: 0.0118879	total: 1m 9s	remaining: 8m 57s
500:	learn: 0.0131861	total: 1m 28s	remaining: 8m 47s
600:	learn: 0.0144383	total: 1m 46s	remaining: 8m 34s
700:	learn: 0.0156167	total: 2m 5s	remaining: 8m 20s
800:	learn: 0.0166911	total: 2m 23s	remaining: 8m 4s
900:	learn: 0.0178537	total: 2m 41s	remaining: 7m 46s
1000:	learn: 0.0189673	total: 3m	remaining: 7m 30s
1100:	learn: 0.0200166	total: 3m 18s	remaining: 7m 12s
1200:	learn: 0.0210992	total: 3m 36s	remaining: 6m 55s
1300:	learn: 0.0220069	total: 3m 55s	remaining: 6m 37s
1400:	learn: 0.0229765	total: 4m 13s	remaining: 6m 19s
1500:	learn: 0.0239747	total: 4m 31s	remaining: 6m 2s
1600:	learn: 0.0249125	total: 4m 50s	remaining: 5m 44s
1700:	learn: 0.0257455	total: 5m 8s	remaining: 5m 26s
1800:	learn: 0.0266035	total: 5m 2

iteration,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_r2,▁▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██████
train_rmse,█▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
iteration,3500
train_r2,0.03748
train_rmse,0.80743
w_train_r2,0.03748


CPU times: user 22min 36s, sys: 1min 29s, total: 24min 6s
Wall time: 11min 57s
